<a href="https://colab.research.google.com/github/aryan619348/IMDB_reviews_NLP/blob/main/IMDB_reviews_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we will be exploring the IMDB dataset https://ai.stanford.edu/~amaas/data/sentiment/, which is a inbuilt data set in Tensorflow Data Services.
The data is split into 25000 samples each for testing and training.
 
The goal is to create a "Embedding" for the reviews, here words and associated words are clustered as vectors in a multi-dimensional space. 
The reviews will be of two categories positive and negative

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteZ5NDIP/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteZ5NDIP/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


Splitting the data into training and testing:

In [3]:
import numpy as np
train_data, test_data = imdb['train'], imdb['test']

In [6]:
training_sentences=[]
training_labels=[]
for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())

In [7]:
testing_sentences = []
testing_labels = []
for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())

In [8]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

Tokenizing the sentences:

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [10]:
vocab_size=10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

In [11]:
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

Creating the neural network model

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           160000    
                                                                 
 flatten (Flatten)           (None, 1920)              0         
                                                                 
 dense (Dense)               (None, 6)                 11526     
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


Training the model

In [14]:
model.fit(padded, training_labels_final, epochs=10, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 6s 7ms/step - loss: 0.5000 - accuracy: 0.7407 - val_loss: 0.3482 - val_accuracy: 0.8469
Epoch 2/10
782/782 [==============================] - 5s 7ms/step - loss: 0.2432 - accuracy: 0.9058 - val_loss: 0.3665 - val_accuracy: 0.8409
Epoch 3/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0977 - accuracy: 0.9738 - val_loss: 0.4424 - val_accuracy: 0.8305
Epoch 4/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0281 - accuracy: 0.9964 - val_loss: 0.5140 - val_accuracy: 0.8286
Epoch 5/10
782/782 [==============================] - 6s 7ms/step - loss: 0.0115 - accuracy: 0.9985 - val_loss: 0.5718 - val_accuracy: 0.8295
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0051 - accuracy: 0.9993 - val_loss: 0.6449 - val_accuracy: 0.8256
Epoch 7/10
782/782 [==============================] - 4s 6ms/step - loss: 0.0019 - accuracy: 0.9998 - val_loss: 0.6885 - val_accuracy: 0.8258
Epoch 

Now lets visualize the embeddings:

In [16]:
e=model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 16)


To plot the embedding we need to reverse our word index as currently the word is the key

In [18]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

Writing the vecotors and metadata to files.. 

In [19]:
import io
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

Downloading the files from collab

In [20]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Now plotting the data on https://projector.tensorflow.org/
The vecs.tsv and meta.tsv can be downloaded from my github

Screenshots of the plotted data are added to the git-folder..